In [1]:
import requests

We try with the first page.

In [2]:
r = requests.get('https://www.linternaute.com/ville/c-villes-les-plus-pauvres')
r

<Response [200]>

In [4]:
import pandas as pd
import json

from bs4 import BeautifulSoup

soup = BeautifulSoup(r.text, 'html.parser')

C:\Users\mathi\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\mathi\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


Informations importantes jusqu'à 259 (260 éléments)

In [19]:
list_infos = [soup.select("td")[x].text for x in range(260)]

for j in range(2,285):
    new_r = requests.get('https://www.linternaute.com/ville/c-villes-les-plus-pauvres/{}'.format(j))
    new_soup = BeautifulSoup(new_r.text, 'html.parser')
    list_infos = list_infos + [new_soup.select("td")[x].text for x in range(260)]

In [22]:
import numpy as np
list_infos_np = np.array(list_infos)

In [31]:
cols = ['RANK','CITY','SCORE','REFERENCE TAX INCOME','UNEMPLOYMENT RATE','RSA BENEFICIARIES',
'Average net monthly salaries',
'Percentage of employees on permanent contracts',
'Number of companies',
'Average wealth tax per household n/a',
'CAF beneficiaries',
'Population','Nothing']

In [37]:
df = pd.DataFrame(data = list_infos_np.reshape((-1,13)),columns = cols)
df.head()

,RANK,CITY,SCORE,REFERENCE TAX INCOME,UNEMPLOYMENT RATE,RSA BENEFICIARIES,Average net monthly salaries,Percentage of employees on permanent contracts,Number of companies,Average wealth tax per household n/a,CAF beneficiaries,Population,Nothing
0,1,Denain,"19,20 / 20",12 878 €,"35,18 %","9,78 %","1 900,41 €","74,20 %",1 573 entreprises,nc,"31,10 %",20 415 hab.,
1,2,Boulogne-sur-Mer,"19,16 / 20",16 981 €,"26,66 %","7,23 %","1 995,09 €","74,94 %",2 524 entreprises,nc,"30,44 %",40 588 hab.,
2,3,Hautmont,"19,11 / 20",16 150 €,"32,36 %","8,34 %","1 967,82 €","77,21 %",476 entreprises,nc,"28,30 %",14 318 hab.,
3,4,Creil,"19,06 / 20",15 421 €,"24,99 %","6,48 %","2 009,37 €","75,70 %",2 221 entreprises,nc,"26,86 %",35 970 hab.,
4,4,Fourmies,"19,06 / 20",16 197 €,"29,81 %","8,88 %","1 969,44 €","76,86 %",487 entreprises,nc,"30,02 %",11 403 hab.,


We now clean the data.

In [35]:
colonnes_good = ['CITY','REFERENCE TAX INCOME',
                 'UNEMPLOYMENT RATE','RSA BENEFICIARIES',
                 'Average net monthly salaries',
                 'Percentage of employees on permanent contracts',
                 'Number of companies', 'Average wealth tax per household n/a',
                 'CAF beneficiaries', 'Population']

In [38]:
df = df[colonnes_good]

In [39]:
for col in df.columns:
    if col != 'VILLE':
        df[col] = df[col].astype(str)
        df[col] = df[col].str.replace('[','')
        df[col] = df[col].str.replace(']','')
        df[col] = df[col].str.replace('<td>','')
        df[col] = df[col].str.replace('</td>','')
        df[col] = df[col].str.replace(' ','')
        df[col] = df[col].str.replace('entreprises','')
        df[col] = df[col].str.replace('€','')
        df[col] = df[col].str.replace(',','.')
        df[col] = df[col].str.replace('hab.','')
        try:
            df[col] = df[col].astype(float)
        except:
            plop = 0
df.head()        

,CITY,REFERENCE TAX INCOME,UNEMPLOYMENT RATE,RSA BENEFICIARIES,Average net monthly salaries,Percentage of employees on permanent contracts,Number of companies,Average wealth tax per household n/a,CAF beneficiaries,Population
0,Denain,12878.0,35.18%,9.78%,1900.41,74.20%,1573.0,nc,31.10%,20415.0
1,Boulogne-sur-Mer,16981.0,26.66%,7.23%,1995.09,74.94%,2524.0,nc,30.44%,40588.0
2,Hautmont,16150.0,32.36%,8.34%,1967.82,77.21%,476.0,nc,28.30%,14318.0
3,Creil,15421.0,24.99%,6.48%,2009.37,75.70%,2221.0,nc,26.86%,35970.0
4,Fourmies,16197.0,29.81%,8.88%,1969.44,76.86%,487.0,nc,30.02%,11403.0


In [41]:
df.to_csv('pauvrete-villes.csv')

The 20 cities with the highest salaries

In [45]:
df_num = df[df['Average net monthly salaries'] != 'nc']

In [51]:
df_ord = df_num.sort_values('Average net monthly salaries',ascending = False)
df_ord.index = range(len(df_ord))

df_ord[["CITY","Average net monthly salaries","Population"]].head(20)

,CITY,Average net monthly salaries,Population
0,Paris,4313.14,2145906.0
1,Orsay,3872.55,15554.0
2,Vernouillet,3871.49,10266.0
3,Clamart,3862.03,53179.0
4,Asnières-sur-Seine,3842.46,88004.0
5,Montrouge,3805.46,47907.0
6,Fontainebleau,3671.73,15903.0
7,Marcq-en-Barœul,3619.73,38604.0
8,Joinville-le-Pont,3593.19,19128.0
9,Écully,3574.84,18789.0


Avezrage salaries in the top 20 cities with highest RSA BENEFICIARIES (the RSA is a basic income for individuals who are below the poverty line).

In [52]:
df_ord_new = df_ord[df_ord['RSA BENEFICIARIES'] != 'nc']

df_ord_new = df_ord_new.sort_values('RSA BENEFICIARIES',ascending=False)[["CITY",\
                                                                          "Average net monthly salaries",\
                                                                          'RSA BENEFICIARIES',\
                                                                         "Population"]]

df_ord_new.index = range(1,len(df_ord_new)+1)

df_ord_new.head(20)

,CITY,Average net monthly salaries,RSA BENEFICIARIES,Population
1,LeMarigot,1935.12,9.99%,3078.0
2,Bessèges,2010.67,9.91%,2766.0
3,Denain,1900.41,9.78%,20415.0
4,Port-de-Bouc,2417.64,9.77%,16290.0
5,Sainte-Foy-la-Grande,1952.94,9.75%,2665.0
6,LeRobert,2303.23,9.68%,21627.0
7,Deshaies,2131.61,9.66%,3963.0
8,Sainte-Anne,2320.67,9.55%,24430.0
9,Lamentin,2420.99,9.52%,17774.0
10,Toulenne,2115.49,9.49%,2779.0


The only city in the top 20 recipients of the RSA with more than 100,000 inhabitants is Perpignan.